In [1]:
import numpy as np
import scipy as sp
from gekko import GEKKO

<h2>HS71 problem benchmark:</h2>

In [2]:
m = GEKKO(remote=False)
x = m.Array(m.Var,4,value=1,lb=1,ub=5)
x1,x2,x3,x4 = x                 # rename variables
x2.value = 5; x3.value = 5      # change guess
m.Equation(np.prod(x)>=25)      # prod>=25
m.Equation(m.sum([xi**2 for xi in x])==40) # sum=40
m.Minimize(x1*x4*(x1+x2+x3)+x3) # objective
m.solve()
print(x)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            1
   Constants    :            0
   Variables    :           10
   Intermediates:            0
   Connections  :            5
   Equations    :            7
   Residuals    :            7
 
 Number of state variables:             10
 Number of total equations: -            7
 Number of slack variables: -            1
 ---------------------------------------
 Degrees of freedom       :              2
 
 solver            3  not supported
 using default solver: APOPT
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0  2.62810E+02  4.00000E+01
    1  1.72196E+01  3.45513E-02
    2 

<h2>Create variables:</h2>

In [19]:
K = 12
a = np.zeros(9)
d = 8
a[2],a[4],a[8] = 1/2,-1,1/8

In [20]:
model = GEKKO(remote=False)
vars = [model.Var(0,lb=0) for _ in range(2*K-1)]
y = [model.Var() for _ in range((K*(K+1))//2)]
eig = model.Var()
E = model.Param(value=0)

In [21]:
constraints = [vars[0] == 1]
for i in range(1,2*K-1,2):
    constraints += [vars[i] == 0]
m = 0
while d + m <= 2 * K - 2:
    t = 4 * (m+1) * E * vars[m]
    if m >= 2:
        t += m * (m * m - 1) * vars[m - 2]
    for n in range(1, d):
        t -= 2 * (n + 2 * m + 2) * a[n] * vars[n + m]
    t /= (2 * a[d] * (d + 2 * m + 2))
    constraints += [vars[d + m] == t]
    m += 1

In [22]:
get_var = lambda i,j: 1 if i == j else y[K+(i*(i-1))//2+j]
get_Mij = lambda i,j: vars[i+j] - eig if i == j else vars[i+j]
for i in range(K):
    for j in range(i,K):
        constraints += [get_Mij(i,j) == sum(get_var(i,k)*y[k]*get_var(j,k) for k in range(i+1))]

In [23]:
model.Equations(constraints)
model.Maximize(eig)

In [24]:
model.solve()

 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :          103
   Intermediates:            0
   Connections  :            0
   Equations    :          106
   Residuals    :          106
 
 Number of state variables:            102
 Number of total equations: -          105
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :             -3
 
 * Warning: DOF <= 0
 solver            3  not supported
 using default solver: APOPT
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0 -7.69231E-02  0.00000E+00
    1  3.51953E+1

Exception: @error: Solution Not Found
